# **Data augmentation: RAG for paragraphs (7 classes)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np

!pip install openai

!pip install langchain
!pip install huggingface_hub
!pip install langchain-community langchain-core

!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q datasets
!pip install -q faiss-cpu
!pip install langchainhub

from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.schema.runnable import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain import hub
from langchain.chat_models import ChatOpenAI

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
import openai

openai_api_key = "..." # put your OpenAI key here

## Load the data:

Load train set:

In [ ]:
f_in = open("drive/MyDrive/data/7-classes/train_set_paragraphs.pkl","rb")
#f_in = open("drive/MyDrive/data/7-classes/full_set_paragraphs.pkl","rb")

data_train = pickle.load(f_in)

f_in.close()

Extract examples of specific classes:

In [ ]:
#data_train_positive = [(data_train[i][1].split('\n\n'),data_train[i][2]) for i in range(len(data_train))] # use all examples
#data_train_positive = [(data_train[i][1].split('\n\n'),data_train[i][2]) for i in range(len(data_train)) if data_train[i][2] == cls]
#data_train_positive = [(data_train[i][1].split('\n\n'),data_train[i][2]) for i in range(len(data_train)) if ((data_train[i][2] == 0) or (data_train[i][2] == 2) or (data_train[i][2] == 3) or (data_train[i][2] == 6))]
#data_train_positive = [(data_train[i][1].split('\n\n'),data_train[i][2]) for i in range(len(data_train)) if ((data_train[i][2] == 0) or (data_train[i][2] == 2))]
data_train_positive = [(data_train[i][1].split('\n\n'),data_train[i][2]) for i in range(len(data_train)) if ((data_train[i][2] == 1) or (data_train[i][2] == 2) or (data_train[i][2] == 5))]
#data_train_positive = [(data_train[i][1].split('\n\n'),data_train[i][2]) for i in range(len(data_train)) if ((data_train[i][2] == 1) or (data_train[i][2] == 2) or (data_train[i][2] == 3) or (data_train[i][2] == 4) or (data_train[i][2] == 5) or (data_train[i][2] == 6))]

In [ ]:
data_train_positive[0:3]

[(['Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)',
   'Block “strengths and challenges”',
   'Please describe your challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains  as of today, e.g.:',
   'Logistics of the required volumes (e.g. storage, regional distribution): This is especially an issue in winter, as the farmers are not allowed to fertilise their fields during winter, but we produce potato products every day. The potato waste therefore needs to be stored during the winter months, but this is not handled internally in our company. We are working together with an external logistics company and they are handling all our wastes from the moment it leaves our biogas plant.'],
  1),
 (['Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)',
   'Block “

## Retrieval augmented generation:

Use external data set from Hugging face:

In [ ]:
dataset_name = "Rahulrayudu/Agro_Dataset_QA_Final"

#page_content_column = "Crop"  # or any other column you're interested in
page_content_column = "Question"  # or any other column you're interested in

# Create a loader instance
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)

# Load the data
data = loader.load()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Final_Crop_QA_Dataset - Sheet1.csv:   0%|          | 0.00/709k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4153 [00:00<?, ? examples/s]

Use external data from the NoAW project:

In [ ]:
# Load list of files:
f = open("drive/MyDrive/data/RAG/filelist.txt", "r")

fileNames = []
for line in f:
    try:
      fName = line.strip()
      if fName != '':
          fileNames.append(fName)

    except ValueError:
        print('Invalid input:',line)

f.close()

In [ ]:
# Load data from multiple files:
#data = []

for file in fileNames:
  loader = TextLoader(file)

  data_file = loader.load()

  data.extend(data_file)

In [ ]:
len(data)

4179

Prepare the data:

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150) # fragment documents into chunks of 1000 characters

docs = text_splitter.split_documents(data)

In [ ]:
len(docs)

4930

Compute the embeddings:

In [ ]:
# Define the path to the pre-trained model you want to use
modelPath = "sentence-transformers/all-MiniLM-l6-v2"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

<ipython-input-13-a34a09b47e7d>:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Define database for storing embeddings:

In [ ]:
db = FAISS.from_documents(docs, embeddings)

Define query retriever:

In [ ]:
retriever = db.as_retriever(search_kwargs={"k": 3})

Define an LLM:

In [ ]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | hub.pull("rlm/rag-prompt")
    | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)
    | StrOutputParser()
)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:323: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
<ipython-input-16-df3318f6b8f7>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  | ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=openai_api_key)


## Generate new examples:

Experiment (augmentation) name:

In [ ]:
experiment = '147'

Generate new examples:

In [ ]:
new_training_set_text = []
new_training_set_labels = []

flag = False
for i in range(len(data_train_positive)):
  if data_train_positive[i][1] == 0:
    if "Questionnaire for " in data_train_positive[i][0][0]:
      request = "You are "+data_train_positive[i][0][0].split("Questionnaire for ")[1]+". "+data_train_positive[i][0][2]
      #request = "Generate a phrase on the following context: "+data_train_positive[i][0][3]
      flag = True
  elif data_train_positive[i][1] == 1:
    request = data_train_positive[i][0][2] + " Mention limitations and barriers"
    flag = True
  elif data_train_positive[i][1] == 2:
    request = "Give your expectations on the following question. "+data_train_positive[i][0][2]
    flag = True
  elif data_train_positive[i][1] == 3:
    request = data_train_positive[i][0][2]+" Discuss market opportunities"
    flag = True
  elif data_train_positive[i][1] == 4:
    if 'valoriz' in data_train_positive[i][0][2]:
      request = data_train_positive[i][0][2]
    else:
      request = data_train_positive[i][0][2]+" Make emphasis on valorization"
    flag = True
  elif data_train_positive[i][1] == 5:
    if "Questionnaire for " in data_train_positive[i][0][0]:
      request = "You are "+data_train_positive[i][0][0].split("Questionnaire for ")[1]+". Briefly describe your company and your position"
    flag = True
  elif data_train_positive[i][1] == 6:
    if "describe your company and your position within the company" in data_train_positive[i][0][2]:
      request = "Which type of residue stream do you deal with in your company? Give sowe examples"
    else:
      request = data_train_positive[i][0][2]+" Make emphasis on type of stream"
    flag = True
  if flag:
    response = rag_chain.invoke(request)
    if not "I don't know" in response:
      new_example = data_train_positive[i][0][0]+"\n\n"+data_train_positive[i][0][1]+"\n\n"+data_train_positive[i][0][2]+"\n\n"+response.strip()
      if new_example not in new_training_set_text:
        new_training_set_text.append(new_example)
        new_training_set_labels.append(data_train_positive[i][1])
    flag = False
  if i % 10 == 0:
    print("Generating",i,"segment")
    print(request)

Generating 0 segment
Please describe your challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains  as of today, e.g.: Mention limitations and barriers
Generating 10 segment
Please describe your challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains  as of today, e.g.: Mention limitations and barriers
Generating 20 segment
lease describe your challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains as of today, e.g.: Mention limitations and barriers
Generating 30 segment
You are arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams). Briefly describe your company and your position
Generating 40 segment
How could these challenges be overcome? Mention limitations and barriers
Generating 50 segment
Give your 

Generated examples:

In [ ]:
new_training_set_text[:5]

["Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residual streams)\n\nBlock “strengths and challenges”\n\nPlease describe your challenges regarding the valorization of your agricultural by-products from apple-/tomato-/grape-/potato processing /brewer’s grains  as of today, e.g.:\n\nThe challenges regarding the valorization of agricultural by-products from apple/tomato/grape/potato processing and brewer's grains include logistical challenges, lack of clear ideas on how to utilize the by-products, and high costs related to collection and transportation. Some respondents prefer burning by-products on the production site due to grape phytoplasma risk, ease of use, low economic cost, and low environmental impact. However, the expensive machines required for collection are a barrier to this valorization route.",
 'Questionnaire for arable farmers/ horticulture businesses/ plant production / food processors (= suppliers of residua

Create new segments:

In [ ]:
data_new = [(-1,new_training_set_text[i],new_training_set_labels[i]) for i in range(len(new_training_set_text))]

data_augmented = data_train + data_new

Some stats:

In [ ]:
print('Class 0:', len([i for i in range(len(data_new)) if data_new[i][2]==0]))
print('Class 1:', len([i for i in range(len(data_new)) if data_new[i][2]==1]))
print('Class 2:', len([i for i in range(len(data_new)) if data_new[i][2]==2]))
print('Class 3:', len([i for i in range(len(data_new)) if data_new[i][2]==3]))
print('Class 4:', len([i for i in range(len(data_new)) if data_new[i][2]==4]))
print('Class 5:', len([i for i in range(len(data_new)) if data_new[i][2]==5]))
print('Class 6:', len([i for i in range(len(data_new)) if data_new[i][2]==6]))
print('Total:', len(data_new))

Class 0: 0
Class 1: 61
Class 2: 66
Class 3: 0
Class 4: 0
Class 5: 8
Class 6: 0
Total: 135


In [ ]:
len(data_augmented)

559

Save results:

In [ ]:
f_out = open("drive/MyDrive/data/7-classes/train_set_paragraphs_augm-"+experiment+".pkl","wb")

pickle.dump(data_augmented,f_out)

f_out.close()